In [1]:
import pandas as pd
import nltk
from keybert._model import KeyBERT

In [2]:
kw_model = KeyBERT()

In [3]:
def split_words(x: pd.DataFrame) -> pd.DataFrame:
    x = x.replace('\'','')
    return x.split(' ')

In [4]:
def remove_keywords(row: pd.DataFrame) -> pd.DataFrame:
    for key in row['key_orig']:
        row['text'] = row['text'].replace(key,'')
    return row

In [5]:
# todo: do we need spec stop words?
spec_stopwords = ['aus', 'australian', 'australia']
def get_tags(x: pd.DataFrame) -> pd.DataFrame:
    keywords = kw_model.extract_keywords(x['text'],
                                     keyphrase_ngram_range=(1, 3),
                                     stop_words='english',
                                     use_mmr=True, 
                                     diversity=0.2)
    if len(keywords) == 0:
        keywords = kw_model.extract_keywords(x['text'], keyphrase_ngram_range=(1, 3), stop_words='english', use_mmr=False)
    return keywords if len(keywords) > 0 else [('[NoneTag]', 0)]

In [6]:
def agg_tags(input_df: pd.DataFrame, target_col: str) -> pd.DataFrame:
    reg_rule = r'\((.*?)\)'
    input_df[target_col] = input_df[target_col].astype(str)

    # Extract and reformat term-weight set
    tmp_df = input_df[target_col].str.extractall(reg_rule).reset_index().reset_index()
    split_result = tmp_df[0].str.split(',', expand=True).rename(columns={0:'key', 
                                                                         1:'value'}).reset_index()
    
    merge_df = tmp_df.merge(split_result, on='index').drop('index',axis=1).rename(columns={'level_0': 'items'})
    del tmp_df, split_result
    
    merge_df['key'] = merge_df['key'].map(split_words)
    mapping_df = merge_df.explode('key')[['items', 'key', 'value']]
    mapping_df['key_orig'] = mapping_df['key']
    mapping_df['key'] = mapping_df['key'].map(nltk.PorterStemmer().stem)
    
    # Compute weight according to each word and ordering
    mapping_df['value'] = mapping_df['value'].astype(float)
    sum_df = mapping_df.groupby(['items','key', 'key_orig'])['value'].sum().reset_index().sort_values(['items','value'], ascending=False)
    
    # Sampling top key
    key_df = sum_df.groupby(['items']).head(1)[['items','key']]
    merge_df = sum_df.merge(key_df, on = ['items', 'key'])
    return merge_df

In [7]:
def get_label_by_iteration(input_df: pd.DataFrame, pk: str, text_col: str, interation = 5):
    input_df[text_col] = input_df[text_col].str.lower()
    input_df = input_df.reset_index(drop=True).reset_index()

    for i in range(interation):
        input_df['raw_result'] = input_df.apply(get_tags, axis=1)
        
        merge_df = agg_tags(input_df, 'raw_result')
        key_df = merge_df.groupby(['items','key'])['key_orig'].apply(lambda x: list(set(x))).reset_index()

        first_df = input_df[['index', 'text']].merge(key_df, left_on='index', right_on='items', how='left')

        input_df = input_df.drop('text', axis=1)
        first_df = first_df[first_df['key_orig'].notna()].apply(remove_keywords, axis=1).rename(columns={'key':f'key_{i}'}).drop(['items', 'key_orig'], axis=1)
        input_df = input_df.merge(first_df, on='index', how='left')
    return input_df

In [11]:
input_df = get_label_by_iteration(rel_df, '_id', 'text')

In [15]:
input_df.to_csv('all_tenders_tag.csv', index=0, encoding='utf-8_sig')

In [16]:
sample = input_df.sample(30)

In [17]:
sample.to_csv('sample_tenders_tag.csv', index=0, encoding='utf-8_sig')

In [213]:
rel_df['_id'].nunique()

2697

In [12]:
input_df[['Title', 'Description', 'key_0','key_1','key_2','key_3', 'key_4']].to_csv('tag_exmaple.csv',encoding='utf-8_sig', index=0)

# Preprocess ----------------------------------------------------------------------------------

In [9]:
rel_df = pd.read_csv('dataset/relevant.csv')

In [10]:
rel_df['text'] = rel_df['Description'] + '.' + rel_df['Title']

In [82]:
rel_df['raw_result'] = rel_df.apply(getTags, axis=1)

In [83]:
# rel_df.to_csv('raw_tenders_tag.csv', index=0)

# Postprocess-------------------------------------------------------------------------------------

In [12]:
raw_df = pd.read_csv('dataset/raw_tenders_tag.csv')
raw_df['text'] = raw_df['text'].str.lower()
raw_df = raw_df.reset_index()

In [13]:
# import re
# re_key_rule = r'\((.*?)\)'
# re_item_rule = r'\'(.*?)\', (.*?)'
# def agg_keys(x):
#     re_result = re.findall(re_rule, x)
#     for i in re_result:
#         result = i.split(', ')
#         print(result)

In [14]:
# rel_df['raw_result'].map(agg_keys)

In [33]:
merge_df = agg_tags(raw_df, 'raw_result')
key_df = merge_df.groupby(['items','key'])['key_orig'].apply(lambda x: list(set(x))).reset_index()
first_df = raw_df.merge(key_df, left_on='index', right_on='items')

In [22]:
first_df = first_df.apply(remove_keywords, axis=1).rename(columns={'key':'key_1'}).drop('items', axis=1)

In [24]:
first_df['first_result'] = first_df.apply(get_tags, axis=1)

In [25]:
first_df = first_df.drop('key_orig', axis=1)

In [26]:
first_df.to_csv('first_tenders_tag.csv', index=0)

In [27]:
merge_df = agg_tags(first_df, 'first_result')
key_df = merge_df.groupby(['items','key'])['key_orig'].apply(lambda x: list(set(x))).reset_index()
second_df = first_df.merge(key_df, left_on='index', right_on='items')
second_df = second_df.apply(remove_keywords, axis=1).rename(columns={'key':'key_1'}).drop('items', axis=1)

In [28]:
second_df

,index,ATM ID,Category,Description,Title,_id,result,text,raw_result,items_x,key_1,first_result,items_y,key_1,key_orig
0,0,ITR 2003/23,Engineering and Research and Technology Based ...,(CETS Ref NOIE00483) Conduct of research relev...,NOIE - National Information Research Proposals,6162aa1fe1b7f5c73e6fdfd0,NaN,(cets ref noie00483) conduct of research relev...,"[('information infrastructure nii', 0.7492), (...",0,inform,"[('national infrastructure nii', 0.7722), ('pr...",0,infrastructur,[infrastructure]
1,1,NS0579RFP,Specialised educational services,The Australian Broadcasting Corporation (ABC) ...,ABC - Project Management Training Services,6162aa1fe1b7f5c73e6fdfd1,NaN,the australian broadcasting corporation () is ...,"[('abc project management', 0.7385), ('service...",1,abc,"[('project management pm', 0.7011), ('manageme...",1,manag,[management]
2,2,NCON/03/89,Management advisory services,Consultancy to provide advice on applications ...,NOIE Business and Technical Consultancy,6162aa1fe1b7f5c73e6fdfd2,NaN,consultancy to provide advice on applications ...,"[('broker projects cbbp', 0.6674), ('coordinat...",2,broker,"[('infrastructure fund ccif', 0.6402), ('based...",2,fund,[fund]
3,3,NS0589EOI,Management advisory services,The ABC is seeking Expressions of Interest (EO...,ABC - BROADCAST & TECHNICAL RELOCATION CONSULT...,6162aa1fe1b7f5c73e6fdfd3,NaN,the is seeking expressions of interest (eoi) ...,"[('technical relocation consultancy', 0.5537),...",3,consult,"[('abc seeking', 0.5644), ('existing premises ...",3,abc,[abc]
4,4,NCON/04/10,Management advisory services,The National Office for the Information Econom...,"ICT, Organisation and Management – The Strateg...",6162aa1fe1b7f5c73e6fdfd4,NaN,the national office for the information econom...,"[('information economy noie', 0.6509), ('consu...",4,manag,"[('ict organisation strategic', 0.6639), ('inf...",4,ict,[ict]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2635,2641,NGTF-MFAPH2-2021,Military science and research,Next Generation Technologies Fund - Multi-Func...,NGTF- Multi-Function Aperture Technology Phase 2,6162aa1fe1b7f5c73e6fea53,NaN,next generation fund - multi-function phase...,"[('function aperture technology', 0.805), ('ge...",2641,apertur,"[('multi function technology', 0.707), ('techn...",2641,technolog,"[technology, technologies]"
2636,2642,FIN/007/20Notice8,Management advisory services,This notice is to inform industry of updates t...,Industry Update – Management Advisory Services...,6162aa1fe1b7f5c73e6fea54,NaN,this notice is to inform of updates to the ti...,"[('advisory services australian', 0.6547), ('i...",2642,advisori,"[('management services australian', 0.6271), (...",2642,industri,[industry]
2637,2643,FIN/007/20Notice1A,Economics,This notice is to inform interested parties ab...,Industry Notice – Management Advisory Services...,6162aa1fe1b7f5c73e6fea55,NaN,this notice is to inform interested parties ab...,"[('advisory services australian', 0.6789), ('p...",2643,advisori,"[('panel industry notice', 0.6652), ('manageme...",2643,manag,[management]
2638,2644,HUB-16-PIN-SIF-001j,Military science and research,"Defence Innovation Hub (Hub), an initiative of...",Defence Innovation Hub - Call For Submissions,6162aa1fe1b7f5c73e6fea57,NaN,"innovation (), an initiative of the australi...","[('defence innovation hub', 0.8319), ('respond...",2644,defenc,"[('innovation hub hub', 0.8231), ('hub initiat...",2644,hub,[hub]


In [10]:
# merge_df[merge_df['items'].isin(merge_df[merge_df['items'].duplicated()]['items'].unique())]

,items,key,key_orig,value
335,2310,applic,applications,0.7071
336,2310,applic,application,0.3885
341,2305,rang,ranges,0.8193
342,2305,rang,range,0.3826
829,1818,engag,engagement,0.7590
830,1818,engag,engage,0.1619
899,1749,valuat,valuation,3.5725
900,1749,valuat,valuations,0.8448
1523,1126,learn,learning,1.4038
1524,1126,learn,learn,0.9690


In [11]:
raw_df.iloc[476]['text']

'A tertiary program aimed at maintaining DITR`s expertise in economics and policy development and developing DITR`s focus on organisational management and leadership.  The program will be delivered during a normal working day i.e. Monday to Friday between 8.30 a.m. and 5.00 p.m.    The Department, through the Human Resource Management Branch, will be responsible for nominating staff for the program and overseeing all phases of program delivery and evaluation including monitoring of performance with payments at milestones.  .Delivery of DITR Post Graduate Program'

In [14]:
cnt_df = pd.DataFrame(merge_df['key'].value_counts())

In [20]:
pd.set_option('display.max_rows', 100)
cnt_df[cnt_df['key']>2]['key'].sum()

1786

# Export-------------------------------------------------------------------------------------

In [35]:
import numpy as np
input_df = pd.read_csv('./dataset/tenders/all_tenders_tag.csv')

In [36]:
input_df[input_df['_id'] == '6162aa1fe1b7f5c73e6fea50']

,index,ATM ID,Category,Description,Title,_id,raw_result,key_0,key_1,key_2,key_3,text,key_4
2688,2688,PROC-9176414,Crop production and management and protection,Branded Publication 2022-2024,Branded Publication 2022-2024,6162aa1fe1b7f5c73e6fea50,"[('[NoneTag]', 0)]",public,brand,2022,2024,-. -,[nonetag]


In [3]:
input_df = input_df[['_id', 'key_0', 'key_1', 'key_2', 'key_3', 'key_4']]

In [9]:
input_df[input_df['key_4'] == '[nonetag]']

,_id,key_0,key_1,key_2,key_3,key_4
22,6162aa1fe1b7f5c73e6fdfe6,provis,servic,train,[nonetag],[nonetag]
24,6162aa1fe1b7f5c73e6fdfe8,pbac,evalu,submiss,extern,[nonetag]
72,6162aa1fe1b7f5c73e6fe018,plan,strateg,acma,develop,[nonetag]
76,6162aa1fe1b7f5c73e6fe01c,intellig,report,region,econom,[nonetag]
104,6162aa1fe1b7f5c73e6fe038,consult,manag,chang,aca,[nonetag]
...,...,...,...,...,...,...
2607,6162aa1fe1b7f5c73e6fe9ff,servic,data,global,lightn,[nonetag]
2615,6162aa1fe1b7f5c73e6fea07,disput,resolut,servic,[nonetag],[nonetag]
2625,6162aa1fe1b7f5c73e6fea11,deliveri,band,se,[nonetag],[nonetag]
2660,6162aa1fe1b7f5c73e6fea34,tonga,resourc,platform,[nonetag],[nonetag]


In [10]:
input_df = input_df.replace('[nonetag]', np.nan)

In [23]:
melt_key = input_df.melt(id_vars = '_id')

In [26]:
melt_key = melt_key.groupby('value')['_id'].agg(len).reset_index().sort_values('_id', ascending=False).rename(columns={'_id':'cnt'})

In [29]:
common_df = melt_key[melt_key['cnt']>2]

In [33]:
cond_1 = input_df['key_0'].isin(common_df['value'].unique())
cond_2 = input_df['key_1'].isin(common_df['value'].unique())
cond_3 = input_df['key_2'].isin(common_df['value'].unique())
cond_4 = input_df['key_3'].isin(common_df['value'].unique())
cond_5 = input_df['key_4'].isin(common_df['value'].unique())
input_df[~((cond_1)|(cond_2)|(cond_3)|(cond_4)|(cond_5))]

,_id,key_0,key_1,key_2,key_3,key_4
848,6162aa1fe1b7f5c73e6fe320,charg,gun,mc,155mm,modular
1587,6162aa1fe1b7f5c73e6fe603,anzac,gallipoli,commemor,ballot,ticket
2259,6162aa1fe1b7f5c73e6fe8a3,refug,brassica,diamondback,moth,infest
2284,6162aa1fe1b7f5c73e6fe8bc,style,manual,dta,7th,edit


In [32]:
tmp[tmp['key']>5].iloc[30:60]

NameError: name 'tmp' is not defined

In [7]:
rel_df['raw_result'] = rel_df['raw_result'].astype(str)

In [8]:
tmp_df = rel_df['raw_result'].str.extractall(r'\((.*?)\)').reset_index().reset_index()

In [9]:
split_result = tmp_df[0].str.split(',', expand=True).rename(columns={0:'key', 1:'value'}).reset_index()

In [10]:
tmp_df = tmp_df.merge(split_result, on='index').drop('index',axis=1).rename(columns={'level_0': 'items'})

In [13]:
tmp_df['key'] = tmp_df['key'].map(split_words)

In [14]:
mapping_df = tmp_df.explode('key')[['items', 'key', 'value']]

In [16]:
mapping_df['key'] = mapping_df['key'].map(nltk.PorterStemmer().stem)

In [18]:
mapping_df['value'] = mapping_df['value'].astype(float)

In [20]:
sum_df = mapping_df.groupby(['items','key'])['value'].sum().reset_index().sort_values(['items','value'], ascending=False)

In [24]:
key_df1 = sum_df.groupby(['items']).head(1)

,items,key,value
22372,2696,indigen,3.6211
22365,2695,hub,4.9894
22355,2694,advisori,1.9368
22347,2693,advisori,1.9368
22337,2692,advisori,1.2605
...,...,...,...
40,4,ict,1.9011
29,3,consult,1.0464
18,2,broker,1.8478
13,1,manag,2.6545
